## Custom API +  Custom Worker

#### Import dependencies

In [1]:
# stdlib
import os

# third party
import requests

# syft absolute
import syft as sy
from syft.service.settings.settings import NodeSettingsUpdate
from syft.service.worker.worker_image import SyftWorkerImage

from getpass import getpass  # noqa

In [2]:
os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "k8s"
os.environ["DEV_MODE"] = "True"

In [3]:
domain_client = sy.login(email="info@openmined.org", password="changethis", port=8080)

ConnectionError: Failed to fetch http://localhost:8080/api/v2/metadata. Response returned with code 502

In [ ]:
domain_client.worker_pools

## Register a custom Image

In [ ]:
registry = os.getenv("SYFT_BASE_IMAGE_REGISTRY", "k3d-registry.localhost:5800")
repo = "openmined/grid-backend"

if "k3d" in registry:
    res = requests.get(url=f"http://{registry}/v2/{repo}/tags/list")
    tag = res.json()["tags"][0]
else:
    tag = sy.__version__

In [4]:
custom_dockerfile_str = f"""
FROM {registry}/{repo}:{tag}

RUN pip install google-cloud-bigquery db_dtypes

""".strip()

NameError: name 'registry' is not defined

In [ ]:
print(custom_dockerfile_str)

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)

In [ ]:
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config
)
submit_result

In [ ]:
dockerfile_list = domain_client.images.get_all()
dockerfile_list

In [ ]:
workerimage = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == custom_dockerfile_str
    ),
    None,
)

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)
workerimage

In [ ]:
external_registry = os.getenv("EXTERNAL_REGISTRY", "k3d-registry.localhost:5800")
external_registry_username = os.getenv("EXTERNAL_REGISTRY_USERNAME", None)
external_registry_password = os.getenv("EXTERNAL_REGISTRY_PASSWORD", None)

In [ ]:
docker_tag = "openmined/custom-worker:0.7.8"

In [ ]:
registry_add_result = domain_client.api.services.image_registry.add(
    "k3d-registry.localhost:5800"
)
registry_add_result

image_registry_list = domain_client.api.services.image_registry.get_all()
image_registry_list

local_registry = image_registry_list[0]
local_registry

local_registry = domain_client.api.services.image_registry.get_all()[0]
registry_uid = local_registry.id

# build with registry_uid
docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=registry_uid,
)
docker_build_result

In [ ]:
image_list = domain_client.images.get_all()
# we can also index with string using the repo_with_tag format
workerimage = next((image for image in image_list if image.id == workerimage.id), None)
assert workerimage is not None, str([image.__dict__ for image in image_list])
assert workerimage.is_built is not None, str(workerimage)
assert workerimage.built_at is not None, str(workerimage)
assert workerimage.image_hash is not None, str(workerimage)
assert image_list[workerimage.built_image_tag] == workerimage

In [ ]:
domain_client.api.services.worker_image.push(
    workerimage.id,
    username="admin",
    password="admin",
)

In [ ]:
worker_pool_name = "custom-pool"
domain_client.api.services.worker_pool.launch(
    name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=3,
    reg_username="admin",
    reg_password="admin",
)

In [ ]:
new_default_worker_pool = NodeSettingsUpdate(default_worker_pool="custom-pool")
domain_client.settings.update(settings=new_default_worker_pool)

In [ ]:
@sy.mock_api_endpoint(settings={"Hello": "Public"})
def public_function(
    context,
) -> str:
    return "Public Function Execution"


@sy.private_api_endpoint(settings={"Hello": "Private"})
def private_function(
    context,
) -> str:
    # =============== Custom Import ===============
    # third party

    return "If this works, is because it's running in custom-pool"


new_endpoint = sy.TwinAPIEndpoint(
    path="first.query",
    mock_function=public_function,
    private_function=private_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)

In [ ]:
domain_client.refresh()

In [ ]:
@sy.syft_function_single_use(
    endpoint=domain_client.api.services.first.query,
)
def job_function(endpoint):
    return endpoint()


new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_client],
)

result = new_project.create_code_request(job_function, domain_client)
domain_client.requests[-1].approve()

In [ ]:
domain_client.settings.get().default_worker_pool

In [ ]:
domain_client.code.job_function(
    endpoint=domain_client.api.services.first.query, blocking=False
)

In [ ]:
domain_client.jobs[0].result